In [3]:
from lumibot.brokers import Alpaca
from lumibot.backtesting import YahooDataBacktesting
from lumibot.strategies.strategy import Strategy
from lumibot.traders import Trader
from datetime import datetime
from alpaca_trade_api import REST
from timedelta import Timedelta

In [4]:
API_KEY = "PK1KAWB732Z95VFMPIEB"
API_SECRET = "lRxdiVh4jOoyZQvgUdqplHNuQZWI9apIPIXPaoaM"
BASE_URL = "https://paper-api.alpaca.markets"
ALPACA_CREDS = {
    "API_KEY":API_KEY,
    "API_SECRET":API_SECRET,
    "PAPER": True
}

In [7]:


class MLTrader(Strategy):
    def initialize(self, symbol:str="SPY", cash_at_risk:float = .5):
        self.cash_at_risk = cash_at_risk
        self.symbol = symbol
        self.sleeptime = "24H"
        self.last_trade = None
        self.api = REST(base_url=BASE_URL, 
                        key_id=API_KEY, 
                        secret_key=API_SECRET)
    
    def position_sizing(self):
        cash = self.get_cash()
        last_price = self.get_last_price(self.symbol)
        quantity = round(cash * self.cash_at_risk / last_price, 0)

        return cash, last_price, quantity
    
    def get_dates(self):
        today = self.get_datetime()
        three_days_prior = today - Timedelta(days = 3)
        return today.strftime('%Y-%m-%d'), three_days_prior.strftime('%Y-%m-%d')
    
    def get_news(self):
        today, three_days_prior = self.get_dates()
        news = self.api.get_news(symbol=self.symbol, 
                                 start = three_days_prior, 
                                 end = today )
        
        news = [ev.__dict__["_raw"]["headline"] for ev in news]
        return news


    def on_trading_iteration(self):
        cash, last_price, quantity = self.position_sizing()
        if cash > last_price:
            if self.last_trade == None:
                news = self.get_news()
                print(news)
                order = self.create_order(
                    self.symbol,
                    quantity,
                    "buy",
                    type="bracket",
                    take_profit_price=last_price*1.20,
                    stop_loss_price=last_price*.95
                )
                self.submit_order(order)
                self.last_trade = "buy"
        


brokers = Alpaca(ALPACA_CREDS)
strategy = MLTrader(name='mlstrat', broker=brokers, parameters={"symbol": "SPY", 
                                                                "cash_at_risk":.5})

start_date = datetime(2023,12,15)
end_date = datetime(2023,12,31)

strategy.backtest(
    YahooDataBacktesting,
    start_date,
    end_date,
    parameters={"symbol": "SPY", 
                "cash_at_risk":.5}
)

Starting backtest for MLTrader...
Progress ||   2.47%  [Elapsed: 0:00:01 ETA: 0:01:07] ["If You Invested $1,000 In Exxon Mobil When Joe Biden Was Elected President, Here's How Much You'd Have Today", 'U.S. Senators Will Vote On Foreign Aid Supplemental Agreement Next Week', 'Traders May Need Frequent Portfolio Adjustments In 2024: BlackRock', 'U.S. Supreme Court Declines To Block Illinois Ban On Assault-Type Rifles And Large Capacity Magazines', "US Stocks Eye 7-Day Winning Streak As Mortgage Rates Fall Below 7%, Yields Tumble: What's Driving Markets Thursday?", 'Live On CNBC, SEC Chair Gary Gensler Discusses Bitcoin ETFs; Regulating Crypto', "Live On CNBC, SEC Chair Gary Gensler Discusses The SEC's Latest Moves; Treasury Clearing; Regulating Artificial Intelligence", "U.S. House Of Representatives Passes $886B Defense Policy Bill 310 To 118, Sending Measure To White House For Biden's Signature", 'Market Clubhouse Morning Memo - December 14th, 2023 (Trade Strategy For SPY, QQQ, AAPL, M

{'cagr': 0.25708368926531966,
 'volatility': 0.028190598556940217,
 'sharpe': 7.2624812234837774,
 'max_drawdown': {'drawdown': 0.0027690901105407078,
  'date': Timestamp('2023-12-21 09:30:00-0500', tz='America/New_York')},
 'romad': 92.84049236488013,
 'total_return': 0.00944036000000037}

## ML Components for news (Sentimental Analysis)

In [ ]:
# pip install alpaca-trade-api==3.1.1
# pip install timedelta